In [1]:
##Part B - Structured Streaming (30 Points)

In [2]:
spark

In [3]:

display(dbutils.fs.ls("/FileStore/tables"))

In [4]:
from pyspark.sql.types import *

In [5]:
inputPath = "dbfs:/FileStore/tables/"

In [6]:
string = "_c"
column_name = [string + str(i) for i in range(0,129,1)]

In [7]:
fields = [StructField( x , StringType(), True) for x in column_name] 

In [8]:
  csvSchema= StructType(fields) #creating schema for the input files

In [9]:
staticInputDF = (
  spark
  #  .read
    .readStream
    .schema(csvSchema)
    .option("maxFilesPerTrigger", 1)
    .csv(inputPath, header=True)
)

display(staticInputDF)
staticInputDF.isStreaming


In [10]:
from pyspark.sql.functions import *
#selecting 50% of data as the readings are taken at 50% overlap (128 readings/window). 
streamingCountsDF = ( 
       staticInputDF
       .select(staticInputDF.columns[:64])
       .drop("_c0")
       
       )

streamingCountsDF.createOrReplaceTempView("Grouped_data")
display(streamingCountsDF)


In [11]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("console")        # memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("counts")     # counts = name of the in-memory table
    #.outputMode("")  # complete = all the counts should be in the table
    .start()
)

In [12]:
from time import sleep
sleep(5)  # wait